In [ ]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from datetime import datetime
import math
from tensorflow.keras.layers import Concatenate, Dense, Activation, Dropout, Input, LSTM, Reshape, Lambda, RepeatVector, TimeDistributed, Flatten
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
from functions import feature_extraction
from tensorflow.keras import backend as K

In [ ]:
AEP = pd.read_csv('AEP_hourly.csv')
AEP = AEP.rename(columns={'AEP_MW': 'Load'})
look_back = 7 #in days
predict = 24 # in hours
df = AEP.copy()


In [ ]:
df['Datetime'] = df['Datetime'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
    
df['Year'] = df['Datetime'].apply(lambda x: x.year)
# featurs of Month_CS from Month

df['cos(M)'] = df['Datetime'].apply(lambda x: np.cos(2*math.pi*x.month/12))
df['sin(M)'] = df['Datetime'].apply(lambda x: np.sin(2*math.pi*x.month/12))

# featurs of Day_CS from Day
df['cos(D)'] = df['Datetime'].apply(lambda x: np.cos(2*math.pi*x.day/31))
df['sin(D)'] = df['Datetime'].apply(lambda x: np.sin(2*math.pi*x.day/31))

# featurs of Weekday_CS from Weekday
df['cos(WD)'] = df['Datetime'].apply(lambda x: np.cos(2*math.pi*x.weekday()/7))
df['sin(WD)'] = df['Datetime'].apply(lambda x: np.sin(2*math.pi*x.weekday()/7))

# featurs of Hour_CS from Hour
df['cos(H)'] = df['Datetime'].apply(lambda x: np.cos(2*math.pi*x.hour/24))
df['sin(H)'] = df['Datetime'].apply(lambda x: np.sin(2*math.pi*x.hour/24))

df = df.drop('Datetime', axis=1)

In [ ]:
# ## df test
# df = pd.DataFrame({'Numbers': range(10),
#                    'A': np.random.rand(10),
#                    'B': np.random.rand(10),
#                    'C': np.random.rand(10)})
# df

In [ ]:
def sliding_window(vector, window_size):
    windows = np.lib.stride_tricks.sliding_window_view(vector, window_size)
    return windows

In [ ]:
n_look_back = 24*21
n_look_forward = 24

data = np.array(df.values)
windows = np.squeeze(sliding_window(data, (n_look_back+n_look_forward,data.shape[1])))
look_back_seq = np.expand_dims(windows[:,:n_look_back,0],axis=2)
look_back_dt = windows[:,:n_look_back,1:]
look_forward_dt = windows[:,-n_look_forward:,1:]
look_forward_seq = np.expand_dims(windows[:,-n_look_forward:,0],axis=2)
print(f'look_back_seq.shape = {look_back_seq.shape}')
print(f'look_back_dt.shape = {look_back_dt.shape}')
print(f'look_forward_seq.shape = {look_forward_seq.shape}')
print(f'look_forward_dt.shape = {look_forward_dt.shape}')